 こちらのライブラリのコードを修正してからインストールが必要 (2020.12.15現在)
* https://github.com/okankop/vidaug
<br/>以下手順
* 適当な場所で git clone https://github.com/okankop/vidaug
* こちらのリンクを参考に該当箇所を修正 https://github.com/okankop/vidaug/pull/13/files
* python setup.py sdist && pip install dist/vidaug-0.1.tar.gz

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from vidaug import augmentors as va
import scipy
import random
%matplotlib inline

In [3]:
folder = "./videos/original"
cropSize = (112, 112)

for curDir, dirs, files in os.walk(folder, topdown=False):
    print(files)
len(files)

['23208523_20190705_4ch.avi', '5705058_20170706_4ch.avi', '20461331_20191009_4ch.avi', '8569030_20170310_4ch.avi', '987107_20170530_4ch.avi', '21516026_20150611_4ch.avi', '22408330_20160905_4ch.avi', '5580519_20190410_4ch.avi', '8770364_20181221_4ch.avi', '21584354_20160118_4ch.avi', '4629440_20190328_4ch.avi', '8321879_20160120_4ch.avi', '22624076_20170803_4ch.avi', '22142251_20180704_4ch.avi', '21774344_20180705_4ch.avi', '8164576_20190605_4ch.avi', '23331218_20191212_4ch.avi', '5632658_20180115_4ch.avi', '22142251_20190611_4ch.avi', '21693262_20150731_4ch.avi', '20615159_20170901_4ch.avi', '7060981_20160810_4ch.avi', '23066053_20181220_4ch.avi', '22803753_20181207_4ch.avi', '6195069_20150812_4ch.avi', '22624076_20180116_4ch.avi', '23287287_20191004_4ch.avi', '20751657_20190111_4ch.avi', '1109057_20190124_4ch.avi', '22976567_20180907_4ch.avi', '1109057_20190619_4ch.avi', '22784045_20190828_4ch.avi', '21410444_20170421_4ch.avi', '20710513_20160516_4ch.avi', '22032270_20160329_4ch.avi'

302

In [4]:
def loadvideo(filename: str) -> np.ndarray:

    if not os.path.exists(filename):
        raise FileNotFoundError(filename)

    capture = cv2.VideoCapture(filename)

    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    v = np.zeros((frame_count, frame_width, frame_height, 3), np.uint8)

    for count in range(frame_count):
        ret, frame = capture.read()
        if not ret:
            raise ValueError("Failed to load frame #{} of {}.".format(count, filename))

        v[count] = frame

#     v = v.transpose((3, 0, 1, 2))

    return v

In [8]:
def transform_video(path, video_aug = "RandomRotate"):
    v = loadvideo(path)
    print(path)
    print(video_aug)
    print(v.shape)
    
    sometimes = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 100% probability
    if video_aug == "RandomRotate":
        seq = va.Sequential([ # randomly rotates the video with a degree randomly choosen from [-10, 10]  
            sometimes(va.RandomRotate((-15, 15)))# horizontally flip the video with 100% probability
        ])
    elif video_aug == "InvertColor":
        seq = va.Sequential([ 
            sometimes(va.InvertColor())
        ])
    elif video_aug == "GaussianBlur":
        seq = va.Sequential([ 
            sometimes(va.GaussianBlur(3.0))
        ])
    elif video_aug == "HorizontalFlip":
        seq = va.Sequential([ 
            sometimes(va.HorizontalFlip())
        ])
    elif video_aug == "Downsample":
        seq = va.Sequential([ 
            sometimes(va.Downsample(0.7))
        ])
    elif video_aug == "Upsample":
        seq = va.Sequential([ 
            sometimes(va.Upsample(1.3))
        ])
    
    frames  = np.array(seq(v))
    print(frames.shape)
    
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    
    output_folder = os.path.join("videos", video_aug)
    if not os.path.exists(output_folder):
            os.mkdir(output_folder)
    
    output_movie = cv2.VideoWriter(os.path.join(output_folder, filename), fourcc, 30, cropSize)
    
    for f in frames:
        output_movie.write(f)
    output_movie.release()

In [9]:
for filename in files:
    path = os.path.join(folder, filename)
    
    # 必要でないものはコメントアウト
    transform_video(path, "RandomRotate")
    transform_video(path, "InvertColor")
    transform_video(path, "GaussianBlur")
    transform_video(path, "HorizontalFlip")
    transform_video(path, "Downsample")
    transform_video(path, "Upsample")

./videos/original/23208523_20190705_4ch.avi
RandomRotate
(50, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/23208523_20190705_4ch.avi
InvertColor
(50, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/23208523_20190705_4ch.avi
GaussianBlur
(50, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/23208523_20190705_4ch.avi
HorizontalFlip
(50, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/23208523_20190705_4ch.avi
Downsample
(50, 112, 112, 3)
(35, 112, 112, 3)
./videos/original/23208523_20190705_4ch.avi
Upsample
(50, 112, 112, 3)
(65, 112, 112, 3)
./videos/original/5705058_20170706_4ch.avi
RandomRotate
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5705058_20170706_4ch.avi
InvertColor
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5705058_20170706_4ch.avi
GaussianBlur
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5705058_20170706_4ch.avi
HorizontalFlip
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5705058_20170706_4ch.avi
Downsample
(52, 112, 112, 3)
(36, 1

./videos/original/8164576_20190605_4ch.avi
GaussianBlur
(60, 112, 112, 3)
(60, 112, 112, 3)
./videos/original/8164576_20190605_4ch.avi
HorizontalFlip
(60, 112, 112, 3)
(60, 112, 112, 3)
./videos/original/8164576_20190605_4ch.avi
Downsample
(60, 112, 112, 3)
(42, 112, 112, 3)
./videos/original/8164576_20190605_4ch.avi
Upsample
(60, 112, 112, 3)
(78, 112, 112, 3)
./videos/original/23331218_20191212_4ch.avi
RandomRotate
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/23331218_20191212_4ch.avi
InvertColor
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/23331218_20191212_4ch.avi
GaussianBlur
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/23331218_20191212_4ch.avi
HorizontalFlip
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/23331218_20191212_4ch.avi
Downsample
(66, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/23331218_20191212_4ch.avi
Upsample
(66, 112, 112, 3)
(85, 112, 112, 3)
./videos/original/5632658_20180115_4ch.avi
RandomRotate
(53, 112, 112, 3)
(53, 112,

(34, 112, 112, 3)
./videos/original/1109057_20190619_4ch.avi
HorizontalFlip
(34, 112, 112, 3)
(34, 112, 112, 3)
./videos/original/1109057_20190619_4ch.avi
Downsample
(34, 112, 112, 3)
(23, 112, 112, 3)
./videos/original/1109057_20190619_4ch.avi
Upsample
(34, 112, 112, 3)
(44, 112, 112, 3)
./videos/original/22784045_20190828_4ch.avi
RandomRotate
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/22784045_20190828_4ch.avi
InvertColor
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/22784045_20190828_4ch.avi
GaussianBlur
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/22784045_20190828_4ch.avi
HorizontalFlip
(72, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/22784045_20190828_4ch.avi
Downsample
(72, 112, 112, 3)
(50, 112, 112, 3)
./videos/original/22784045_20190828_4ch.avi
Upsample
(72, 112, 112, 3)
(93, 112, 112, 3)
./videos/original/21410444_20170421_4ch.avi
RandomRotate
(65, 112, 112, 3)
(65, 112, 112, 3)
./videos/original/21410444_20170421_4ch.avi
InvertColor
(65, 112

(42, 112, 112, 3)
./videos/original/21061841_20170517_4ch.avi
Upsample
(60, 112, 112, 3)
(78, 112, 112, 3)
./videos/original/22117949_20160128_4ch.avi
RandomRotate
(56, 112, 112, 3)
(56, 112, 112, 3)
./videos/original/22117949_20160128_4ch.avi
InvertColor
(56, 112, 112, 3)
(56, 112, 112, 3)
./videos/original/22117949_20160128_4ch.avi
GaussianBlur
(56, 112, 112, 3)
(56, 112, 112, 3)
./videos/original/22117949_20160128_4ch.avi
HorizontalFlip
(56, 112, 112, 3)
(56, 112, 112, 3)
./videos/original/22117949_20160128_4ch.avi
Downsample
(56, 112, 112, 3)
(39, 112, 112, 3)
./videos/original/22117949_20160128_4ch.avi
Upsample
(56, 112, 112, 3)
(72, 112, 112, 3)
./videos/original/5632658_20151218_4ch.avi
RandomRotate
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/5632658_20151218_4ch.avi
InvertColor
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/5632658_20151218_4ch.avi
GaussianBlur
(46, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/5632658_20151218_4ch.avi
HorizontalFlip
(46, 11

(58, 112, 112, 3)
./videos/original/5632658_20191017_4ch.avi
HorizontalFlip
(58, 112, 112, 3)
(58, 112, 112, 3)
./videos/original/5632658_20191017_4ch.avi
Downsample
(58, 112, 112, 3)
(40, 112, 112, 3)
./videos/original/5632658_20191017_4ch.avi
Upsample
(58, 112, 112, 3)
(75, 112, 112, 3)
./videos/original/21516026_20181113_4ch.avi
RandomRotate
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/21516026_20181113_4ch.avi
InvertColor
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/21516026_20181113_4ch.avi
GaussianBlur
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/21516026_20181113_4ch.avi
HorizontalFlip
(66, 112, 112, 3)
(66, 112, 112, 3)
./videos/original/21516026_20181113_4ch.avi
Downsample
(66, 112, 112, 3)
(46, 112, 112, 3)
./videos/original/21516026_20181113_4ch.avi
Upsample
(66, 112, 112, 3)
(85, 112, 112, 3)
./videos/original/21158019_20191210_4ch.avi
RandomRotate
(53, 112, 112, 3)
(53, 112, 112, 3)
./videos/original/21158019_20191210_4ch.avi
InvertColor
(53, 112

...

(35, 112, 112, 3)
./videos/original/20781811_20181016_4ch.avi
HorizontalFlip
(35, 112, 112, 3)
(35, 112, 112, 3)
./videos/original/20781811_20181016_4ch.avi
Downsample
(35, 112, 112, 3)
(24, 112, 112, 3)
./videos/original/20781811_20181016_4ch.avi
Upsample
(35, 112, 112, 3)
(45, 112, 112, 3)
./videos/original/5580519_20170606_4ch.avi
RandomRotate
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5580519_20170606_4ch.avi
InvertColor
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5580519_20170606_4ch.avi
GaussianBlur
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5580519_20170606_4ch.avi
HorizontalFlip
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/5580519_20170606_4ch.avi
Downsample
(52, 112, 112, 3)
(36, 112, 112, 3)
./videos/original/5580519_20170606_4ch.avi
Upsample
(52, 112, 112, 3)
(67, 112, 112, 3)
./videos/original/22240592_20160114_4ch.avi
RandomRotate
(35, 112, 112, 3)
(35, 112, 112, 3)
./videos/original/22240592_20160114_4ch.avi
InvertColor
(35, 112, 1

(60, 112, 112, 3)
./videos/original/21238894_20160721_4ch.avi
HorizontalFlip
(60, 112, 112, 3)
(60, 112, 112, 3)
./videos/original/21238894_20160721_4ch.avi
Downsample
(60, 112, 112, 3)
(42, 112, 112, 3)
./videos/original/21238894_20160721_4ch.avi
Upsample
(60, 112, 112, 3)
(78, 112, 112, 3)
./videos/original/5580519_20180605_4ch.avi
RandomRotate
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20180605_4ch.avi
InvertColor
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20180605_4ch.avi
GaussianBlur
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20180605_4ch.avi
HorizontalFlip
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20180605_4ch.avi
Downsample
(49, 112, 112, 3)
(34, 112, 112, 3)
./videos/original/5580519_20180605_4ch.avi
Upsample
(49, 112, 112, 3)
(63, 112, 112, 3)
./videos/original/7914328_20170920_4ch.avi
RandomRotate
(65, 112, 112, 3)
(65, 112, 112, 3)
./videos/original/7914328_20170920_4ch.avi
InvertColor
(65, 112, 112

(64, 112, 112, 3)
./videos/original/4285532_20171117_4ch.avi
HorizontalFlip
(64, 112, 112, 3)
(64, 112, 112, 3)
./videos/original/4285532_20171117_4ch.avi
Downsample
(64, 112, 112, 3)
(44, 112, 112, 3)
./videos/original/4285532_20171117_4ch.avi
Upsample
(64, 112, 112, 3)
(83, 112, 112, 3)
./videos/original/22357362_20160705_4ch.avi
RandomRotate
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/22357362_20160705_4ch.avi
InvertColor
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/22357362_20160705_4ch.avi
GaussianBlur
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/22357362_20160705_4ch.avi
HorizontalFlip
(52, 112, 112, 3)
(52, 112, 112, 3)
./videos/original/22357362_20160705_4ch.avi
Downsample
(52, 112, 112, 3)
(36, 112, 112, 3)
./videos/original/22357362_20160705_4ch.avi
Upsample
(52, 112, 112, 3)
(67, 112, 112, 3)
./videos/original/7043060_20161118_4ch.avi
RandomRotate
(67, 112, 112, 3)
(67, 112, 112, 3)
./videos/original/7043060_20161118_4ch.avi
InvertColor
(67, 112, 

(55, 112, 112, 3)
./videos/original/20254090_20171013_4ch.avi
HorizontalFlip
(55, 112, 112, 3)
(55, 112, 112, 3)
./videos/original/20254090_20171013_4ch.avi
Downsample
(55, 112, 112, 3)
(38, 112, 112, 3)
./videos/original/20254090_20171013_4ch.avi
Upsample
(55, 112, 112, 3)
(71, 112, 112, 3)
./videos/original/5580519_20160601_4ch.avi
RandomRotate
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20160601_4ch.avi
InvertColor
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20160601_4ch.avi
GaussianBlur
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20160601_4ch.avi
HorizontalFlip
(49, 112, 112, 3)
(49, 112, 112, 3)
./videos/original/5580519_20160601_4ch.avi
Downsample
(49, 112, 112, 3)
(34, 112, 112, 3)
./videos/original/5580519_20160601_4ch.avi
Upsample
(49, 112, 112, 3)
(63, 112, 112, 3)
./videos/original/22624076_20190813_4ch.avi
RandomRotate
(58, 112, 112, 3)
(58, 112, 112, 3)
./videos/original/22624076_20190813_4ch.avi
InvertColor
(58, 112, 1

(47, 112, 112, 3)
./videos/original/22141774_20160107_4ch.avi
HorizontalFlip
(47, 112, 112, 3)
(47, 112, 112, 3)
./videos/original/22141774_20160107_4ch.avi
Downsample
(47, 112, 112, 3)
(32, 112, 112, 3)
./videos/original/22141774_20160107_4ch.avi
Upsample
(47, 112, 112, 3)
(61, 112, 112, 3)
./videos/original/21083431_20160413_4ch.avi
RandomRotate
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/21083431_20160413_4ch.avi
InvertColor
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/21083431_20160413_4ch.avi
GaussianBlur
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/21083431_20160413_4ch.avi
HorizontalFlip
(54, 112, 112, 3)
(54, 112, 112, 3)
./videos/original/21083431_20160413_4ch.avi
Downsample
(54, 112, 112, 3)
(37, 112, 112, 3)
./videos/original/21083431_20160413_4ch.avi
Upsample
(54, 112, 112, 3)
(70, 112, 112, 3)
./videos/original/21510102_20190531_4ch.avi
RandomRotate
(59, 112, 112, 3)
(59, 112, 112, 3)
./videos/original/21510102_20190531_4ch.avi
InvertColor
(59, 